# Choice Learn
El presente notebook tiene como objetivo replicar los experimentos relacionados al dataset SwissMetro estudiado en el paper relativo a L-MNL, entendiendo así las características y herramientas propias que la librería ofrece.

In [3]:
from choice_learn.datasets import load_swissmetro
from choice_learn.data import ChoiceDataset 
import pandas as pd

In [4]:
#importamos los datos de swissmetro desde el repositorio del paper
test_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_test.dat", sep="\t")
train_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_train.dat", sep="\t")

In [5]:
# veamos las dimensiones de los datasets y luego sus primeras 5 filas
print(test_df.shape)
print(train_df.shape)

(2145, 28)
(8574, 28)


In [6]:
test_df.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,310,3,0,6,1,1,3,...,103,2600,30,57,4160,20,0,0,0,1
1,3,1,1,950,4,0,1,1,0,2,...,336,65,30,84,97,20,0,416,170,1
2,2,0,1,384,3,1,7,1,1,5,...,148,2610,120,107,3480,30,0,0,0,1
3,3,1,1,989,4,0,1,1,1,3,...,158,38,60,116,57,30,0,110,65,1
4,3,1,1,946,4,0,1,1,0,2,...,149,34,30,66,43,20,0,117,55,1


In [ ]:
train_df.head()

In [ ]:
# el siguiente código es para filtrar el dataframe de la manera que se hace en el paper, es decir,
# se eliminan las filas donde algún individuo no cuente con alguna de las alternativas de transporte
# lo que asegura que cada persona tiene a su disposición las 3 alternativas de transporte


test_df = test_df.loc[test_df.CAR_AV == 1]
test_df = test_df.loc[test_df.SM_AV == 1]
test_df = test_df.loc[test_df.TRAIN_AV == 1]

train_df = train_df.loc[train_df.CAR_AV == 1]
train_df = train_df.loc[train_df.SM_AV == 1]
train_df = train_df.loc[train_df.TRAIN_AV == 1]

In [ ]:
# Se normaliza los valores por un factor de 100 según se estipula en el paper, esto es para evitar problemas de convergencia de la DNN
# esta práctica es usual en ML, ya que los valores muy grandes pueden afectar el entrenamiento de la red. Usualmente se elige escalar
# los valores por max-min, o Q3-Q1, por la desviación estándar u otro método, pero en este caso se elige escalarlos por 100

# Normalizing values by 100
train_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] = (
    train_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] / 100.0
)

train_df[["TRAIN_HE", "SM_HE"]] = (
    train_df[["TRAIN_HE", "SM_HE"]] / 100.0
)

train_df["train_free_ticket"] = train_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)
train_df["sm_free_ticket"] = train_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)

train_df["TRAIN_travel_cost"] = train_df.apply(
    lambda row: (row["TRAIN_CO"] * (1 - row["train_free_ticket"])) / 100, axis=1
)
train_df["SM_travel_cost"] = train_df.apply(
    lambda row: (row["SM_CO"] * (1 - row["sm_free_ticket"])) / 100, axis=1
)
train_df["CAR_travel_cost"] = train_df.apply(lambda row: row["CAR_CO"] / 100, axis=1)


# Normalizing values by 100
test_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] = (
    test_df[["TRAIN_TT", "SM_TT", "CAR_TT"]] / 100.0
)

test_df[["TRAIN_HE", "SM_HE"]] = (
    test_df[["TRAIN_HE", "SM_HE"]] / 100.0
)

test_df["train_free_ticket"] = test_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)
test_df["sm_free_ticket"] = test_df.apply(
    lambda row: (row["GA"] == 1).astype(int), axis=1
)

test_df["TRAIN_travel_cost"] = test_df.apply(
    lambda row: (row["TRAIN_CO"] * (1 - row["train_free_ticket"])) / 100, axis=1
)
test_df["SM_travel_cost"] = test_df.apply(
    lambda row: (row["SM_CO"] * (1 - row["sm_free_ticket"])) / 100, axis=1
)
test_df["CAR_travel_cost"] = test_df.apply(lambda row: row["CAR_CO"] / 100, axis=1)


In [ ]:
# esta parte es más bien técnica, se convierten las columnas a float32 para evitar problemas de memoria
# en el entrenamiento de la red, esto es porque float32 es un tipo de dato que ocupa la mitad de memoria que float64
# y en este caso no se requiere de tanta precisión en los valores

train_df.SM_SEATS = train_df.SM_SEATS.astype("float32")
test_df.SM_SEATS = test_df.SM_SEATS.astype("float32")


# Se resta 1 a la columna CHOICE para que los valores vayan de 0 a 2 en lugar de 1 a 3, que es lo requerido para el modelo

train_df.CHOICE = train_df.CHOICE - 1
test_df.CHOICE = test_df.CHOICE - 1

La siguiente parte es clave, para ello véamosla con un poco más de detalle.

Creación de conjuntos de datos para el modelo:

```python

dataset = ChoiceDataset.from_single_wide_df(...)

train_dataset = ChoiceDataset.from_single_wide_df(...)

test_dataset = ChoiceDataset.from_single_wide_df(...) 
```

- ``ChoiceDataset.from_single_wide_df``: Convierte los DataFrames en objetos ChoiceDataset necesarios para el modelo. 

Sus parámetros son:

- ``choices_column="CHOICE"``: Indica la columna que contiene la elección realizada.

- ``items_id=["TRAIN", "SM", "CAR"]``: Especifica las opciones de transporte disponibles (CAR, TRAIN y SM).
- ``shared_features_columns``: Lista de columnas con características compartidas entre todas las opciones.
- ``items_features_suffixes``: Sufijos que identifican las características específicas de cada opción (tiempo de viaje, costo de viaje, frecuencia de servicio).
- ``choice_format="items_index"``: Especifica el formato en que se almacenan las elecciones.

Como se observa en el código, esto se realizará esto tanto para train como para test.

In [ ]:
train_dataset = ChoiceDataset.from_single_wide_df(df=train_df, choices_column="CHOICE", items_id=["TRAIN", "SM", "CAR"],
shared_features_columns=["GA", "AGE", "LUGGAGE", "SM_SEATS", 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
items_features_suffixes=["TT", "travel_cost", "HE"], choice_format="items_index")

test_dataset = ChoiceDataset.from_single_wide_df(df=test_df, choices_column="CHOICE", items_id=["TRAIN", "SM", "CAR"],
shared_features_columns=["GA", "AGE", "LUGGAGE", "SM_SEATS", 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
items_features_suffixes=["TT", "travel_cost", "HE"], choice_format="items_index")

In [ ]:
# veamos ahora las dimensiones de los datasets resultantes
print(len(train_dataset), len(test_dataset))

In [ ]:
# una vez preprocesada la data, se puede proceder a entrenar el modelo Learning MNL de la manera descrita en el paper,
# esto es, con 200 épocas, optimizador Adam, learning rate de 0.005, y una red neuronal con una capa oculta de 200 neuronas, sin fijar la semilla
from choice_learn.models.learning_mnl import LearningMNL

swiss_model = LearningMNL(optimizer="Adam", lr=0.005,
nn_features=['PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'], nn_layers_widths=[200], epochs=200, batch_size=32) 

#En el paper se menciona qué coeficientes son compartidos y específicos, por ello se aclara en el modelo

# se puede especificar los coeficientes compartidos y específicos de la siguiente manera
swiss_model.add_shared_coefficient(feature_name="TT", items_indexes=[0, 1, 2])  # coeficiente compartido
swiss_model.add_shared_coefficient(feature_name="travel_cost", items_indexes=[0, 1, 2])
swiss_model.add_shared_coefficient(feature_name="HE", items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="GA", items_indexes=[0, 1])
swiss_model.add_shared_coefficient(feature_name="AGE", items_indexes=[0]) # coeficiente específico
swiss_model.add_shared_coefficient(feature_name="LUGGAGE", items_indexes=[2])
swiss_model.add_shared_coefficient(feature_name="SM_SEATS", items_indexes=[1])
swiss_model.add_coefficients(feature_name="intercept", items_indexes=[1, 2])

In [ ]:
# se entrena el modelo con el dataset de entrenamiento y se valida con el dataset de prueba,
# el siguiente código demora al rededor de 20 minutos en correr
hist = swiss_model.fit(train_dataset, val_dataset=test_dataset, verbose=1)

In [ ]:
# esta celda de código es prescindible, se presenta para mostrar como se puede reentrenar el modelo, en este caso con un learning rate menor
# lo que permite ajustar con mayor presición los parámetros a entrenar
swiss_model.assign_lr(0.001)
hist2 = swiss_model.fit(train_dataset, val_dataset=test_dataset, verbose=1)

In [ ]:
# se utiliza un batch_size de 32 para evitar problemas de memoria, y se retorna el error medio de todas las muerstras
swiss_model.evaluate(train_dataset, batch_size=32)  

In [ ]:
swiss_model.evaluate(test_dataset, batch_size=32) 

In [ ]:
# Se muestra los parámetros estimados por el modelo luego del entrenamiento
swiss_model.trainable_weights[:8]

In [ ]:
# veamos el accuracy del modelo en el conjunto de prueba
probas = swiss_model.predict_probas(test_dataset, batch_size=32)

#tenemos las probabilidades de que cada alternativa sea escogida, por lo que diremos que se escoge la alternativa con mayor probabilidad

probas = probas.numpy()
choices = probas.argmax(axis=1)
test_labels = test_dataset.choices

# se calcula el accuracy del modelo
accuracy = (choices == test_labels).mean()
print(f"accuracy: {accuracy}")
# se calcula la precision
precision = ((choices == 1) & (test_labels == 1)).sum() / (choices == 1).sum()
print(f"precision: {precision}")